# Analyzing Blood Bowl kick-off formations using FUMBBL replay data

the botbowl ppl are discussing writing a parser for the json that would result in log with chess like notation ie P1 - B - E4 to C5 - (player one chooses blitz, moves from position e4 to c5 etc.
Currently my knowledge is too limited to contribute to such an endeavour.

I have however cooked up a new FUMBBL data analysis project using replays. 
To learn how to work with the replay files, I want to try and extract the set-up formations for a high stakes tournament (Road to Malta, or the Tilean Team Cup).  
With the data I like to develop a nice viz, working with board positions together with the roster. 
When that goes somewhere I will scale it up to order 100, and try to make plots that either stack / aggregate formations, or do a cluster analysis on the start up formations, and see what can be learned from all of this 🙂 

# What has already been done

Christian Huber (aka Candlejack) seems our man here. He has two repo's open that are of great interest.

First is https://github.com/SanityResort/htmlreplay

This connect to the FFB server and requests a replay file that it reads in and converts to JSON.
The goal seems to be to be able to visualize a replay in the browser via HTML.

Then there is https://github.com/SanityResort/FFBStats
This is code that processes replay files and extracts information from them. Exactly what we want as well!
It was written in 2016 and integrated into the site in 2017 by Christer. They went away and came back in 2022.
It produces a match statistics file as JSON, that forms the basis of a nice visualization on the match result page on FUMBBL.
The match statistics are available through the API.

https://fumbbl.com/p/match?op=stats&id=3916966

# FUMBBL Replay datafiles: opening up the black box

Replay data is quite verbose and there’s lots of it. The FFB client communicates with the Server using Java Web Sockets.
The raw data packages send over the line are web sockets. 
A replay file would be the json command stream. It’s a complex format, as it’s more or less just logging the data packets between the client and server. 

For example, after unzipping, opening as JSON in VScode and doing autoformatting, we end up with a 266K lines of client server "command" stream.
Each turn is about 10K lines, with roster info at the end.

The high level file format is as follows:

```
{
    "gameStatus": "uploaded",
    "stepStack": {
        "steps": []
    },
    "gameLog": {}, # contains the command stream
    "game": {}, # contains the full roster and position information
    "playerIds": [],
    "swarmingPlayerActual": 0,
    "passState": {},
    "prayerState": {},
    "activeEffects": {}
}
```

A full history of all the events during the game is stored under `gameLog`.

The basic unit is the command, that is indexed by `commandNr`. 
A match consists of several thousand commands.
A typical command has the following **FIXED** structure:

```
{
    "netCommandId": "serverModelSync",
    "commandNr": 243,
    "modelChangeList": {
        "modelChangeArray": []
    },
    "reportList": {
        "reports": []
    },
    "sound": null,
    "gameTime": 805789,
    "turnTime": 179506
}
```

`modelChange` changes the game state.
`reportList` directs output to the client's reporting panel.

This could be the basis of our flat file format.
We write a for loop that cycles through the commands, and fills out a pandas dataframe.
We use what we learned with the API data.

Columns in our initial data format:

* commandNr
* modelChangeId
* modelChangeKey
* playerId
* playerState
* playerXcoordinate
* playerYcoordinate
* gameTime
* turnTime

if `modelChangeId` equals `fieldModelSetPlayerState` we record the `modelChangeValue` under `playerState`, and if it equals `fieldModelSetPlayerCoordinate` we record the `modelChangeValue` vector under `playerXcoordinate` and `playerYcoordinate`. In both cases the `PlayerId` can be found under `modelChangeKey`.



## The Field model including the coordinate system

FFB uses a field model that is very straightforward. The 15 x 26 game board is indexed using (X,Y) coordinates, with the top left square being (0,0). 
and the lower right square being (25, 14). 

Players can be either:
* On the pitch
* In the reserve box
* IN the KO box
* In the Badly hurt box
* In the Seriously injured box
* In the RIP box
* In the Ban box
* In The miss next game box

On the pitch the X,Y coordinates are used, the other locations are indexed using 0,1,2 etc. 

```
export default class Coordinate {
    x: number;
    y: number;

    public static FIELD_WIDTH = 26;
    public static FIELD_HEIGHT = 15;
    public static RSV_HOME_X = -1;
    public static KO_HOME_X = -2;
    public static BH_HOME_X = -3;
    public static SI_HOME_X = -4;
    public static RIP_HOME_X = -5;
    public static BAN_HOME_X = -6;
    public static MNG_HOME_X = -7;
    public static RSV_AWAY_X = 30;
    public static KO_AWAY_X = 31;
    public static BH_AWAY_X = 32;
    public static SI_AWAY_X = 33;
    public static RIP_AWAY_X = 34;
    public static BAN_AWAY_X = 35;
    public static MNG_AWAY_X = 36;
    
}
```

FFB uses `fieldModelSetPlayerCoordinate` to position players on the field or on the dug out. Players are identified using the FUMBBL player ids. At the end of the replay, all the player information is stored, including extra skills above those that come with the positional, as well as the full rosters (including all possible star players).

# The phases of the match

The various phases of the match are clearly distinguished in the command streams.
`turnDataSetTurnNr` , `turnDataSetFirstTurnAfterKickoff`, `gameSetTurnMode`, 

# Approach

Using the API I have downloaded a gzipped replay file.
For this match:
https://fumbbl.com/FUMBBL.php?page=match&id=4444067

To goal is to programmatically extract the setup formations.
Here I show my own setup formation.

# Position extraction

# Visualization

We can go for a schematic approach, or try to mimic FFB by plotting FFB icons over the pitch, or a mix, where we plot Letters or codes over the pitch.
In python there is Pillow, the Python image library.
https://randomgeekery.org/post/2017/11/drawing-grids-with-python-and-pillow/

# References

Planning in the midst of chaos: how a stochastic Blood Bowl model can help to identify key planning features


We also need the PlayerState.

export default class PlayerState {
    public static UNKNOWN = 0;
    public static STANDING = 1;
    public static MOVING = 2;
    public static PRONE = 3;
    public static STUNNED = 4;
    public static KNOCKED_OUT = 5;
    public static BADLY_HURT = 6;
    public static SERIOUS_INJURY = 7;
    public static RIP = 8;
    public static RESERVE = 9;
    public static MISSING = 10;
    public static FALLING = 11;
    public static BLOCKED = 12;
    public static BANNED = 13;
    public static EXHAUSTED = 14;
    public static BEING_DRAGGED = 15;
    public static PICKED_UP = 16;
    public static HIT_BY_FIREBALL = 17;
    public static HIT_BY_LIGHTNING = 18;
    public static HIT_BY_BOMB = 19;
    public static BIT_ACTIVE = 256;
    public static BIT_CONFUSED = 512;
    public static BIT_ROOTED = 1024;
    public static BIT_HYPNOTIZED = 2048;
    public static BIT_BLOODLUST = 4096;
    public static BIT_USED_PRO = 8192
}

New plan is to create a data structure (JSON) that holds all player positions and use that to construct / plot the sequential board positions.


In [ ]:
import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd

import gzip
import json

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
def fetch_replay(replay_id):
    
    target = 'raw/df_replay_' + time.strftime("%Y%m%d_%H%M%S") + '.h5'

    print('fetching replay data as JSON')

    dirname = "raw/replay_files/" 
    fname_string_gz = dirname + str(replay_id) + ".gz"        
        
    # PM read compressed json file
    with gzip.open(fname_string_gz, mode = "rb") as f:
        replay = json.load(f)

    return replay

In [ ]:
my_replay = fetch_replay(1602344)



In [ ]:
modelChangeId = []
modelChangeKey = []
modelChangeValue = []
SetPlayerCoordinate = []
PlayerCoordinateX = []
PlayerCoordinateY = []
commandNr = []
turnNr = []
TurnCounter = 0

my_gamelog = my_replay['gameLog']

ignoreList = ['fieldModelAddPlayerMarker', 
              'fieldModelRemoveSkillEnhancements',
              'fieldModelAddDiceDecoration',
              'fieldModelRemoveDiceDecoration',
              'fieldModelAddPushbackSquare',
              'fieldModelRemovePushbackSquare',
              'playerResultSetTurnsPlayed', # we can ignore all playerResult* these are all in game statistic counters
              'playerResultSetBlocks',
              'actingPlayerSetStrength',
              'gameSetConcessionPossible'
              ]

for commandIndex in range(len(my_gamelog['commandArray'])):
    tmpCommand = my_gamelog['commandArray'][commandIndex]
    for modelChangeIndex in range(len(tmpCommand['modelChangeList']['modelChangeArray'])):
        tmpChange = tmpCommand['modelChangeList']['modelChangeArray'][modelChangeIndex]
        if str(tmpChange['modelChangeId']) not in ignoreList:
            if str(tmpChange['modelChangeId']) == 'turnDataSetTurnNr':
                TurnCounter += 1
            turnNr.append(TurnCounter)
            commandNr.append(tmpCommand['commandNr'])
            modelChangeId.append(tmpChange['modelChangeId'])
            modelChangeKey.append(tmpChange['modelChangeKey'])
            modelChangeValue.append(tmpChange['modelChangeValue'])
            if str(tmpChange['modelChangeId']) == "fieldModelSetPlayerCoordinate":
                SetPlayerCoordinate.append(1)
                PlayerCoordinateX.append(tmpChange['modelChangeValue'][0])
                PlayerCoordinateY.append(tmpChange['modelChangeValue'][1])
            else:
                SetPlayerCoordinate.append(0)
                PlayerCoordinateX.append(99)
                PlayerCoordinateY.append(99)

df = pd.DataFrame( {"commandNr": commandNr, 
                    "turnNr": turnNr,
                    "modelChangeId": modelChangeId,
                    "modelChangeKey": modelChangeKey,
                    "modelChangeValue": modelChangeValue,
                    "SetPlayerCoordinate": SetPlayerCoordinate,
                    "PlayerCoordinateX": PlayerCoordinateX,
                    "PlayerCoordinateY": PlayerCoordinateY})

df.to_excel("output.xlsx")  

#df.query('turnNr == 16')


Lets move on to plotting the board state.

We have an empty image of the board as jpg.
We want to plot player icons on it.
